# PyCity Schools Analysis

This analysis of PyCity Schools provides various insights both at the district level and at the school level.

## District Summary
The District Summary analysis (contained in the district_summary dataframe) provides an overview of the district's key metrics: total schools, total students, total budget, average math score, average reading score, percentage of students who passed math, percentage of students who passed reading, and percentage of students who passed both math and reading. This overview shows that while the average scores and the individual passing rates for math and reading are high, the percentage of students who passed both math and reading is relatively low.

## School Summary
The School Summary analysis (contained in the per_school_summary dataframe) provides a summary of key metrics for each school. This includes the type of school, its total number of students, its total budget and budget per student, the averages of math and reading scores, and the percentage of students passing math, reading, and math and reading. 
### Highest Performing and Lowest Performing Schools
From this summary, a further analysis was done to identify the top 5 and the bottom 5 performing schools, based on the percentage of students who are passing both math and reading. The top 5 schools have various budgets and total students, but the key trend is that they are all charter schools (as opposed to district schools). Meanwhile, the bottom 5 schools have significantly higher budgets and a greater total number of students. The 5 bottom schools are all district schools (as opposed to charter schools). This would indicate that charter schools perform better than district schools.
### Math and Reading Scores by Grade
The math and reading scores were also analyzed by grade to identify trends across each school. Students from each school generally have similar average scores across all grades. These results are summarized in two dataframes: math_scores_by_grade and reading_scores_by_grade.

## Scores by School Spending
This analysis categorizes each school's per student budget into 4 bins, or ranges of spending, to then analyze the average scores and passing rates based on the school's spending. The results are summarized in the spending_summary dataframe and show that lower spending schools tend to have higher average scores and passing rates, which would indicate that more spending does not necessarily correlate with higher scores or passing rates.

## Scores by School Size
This analysis categorizes each school's size into 3 bins, or ranges of size, to then analyze the average scores and passing rates based on the school's size. The results are summarized in the size_summary dataframe and show that smaller and medium schools of less than 2,000 students have higher averages and passing rates than large schools of over 2,000 students.

## Scores by School Type
This final analysis compares the average scores and the passing rates of charter schools and district schools. The results are summarized in the type_summary dataframe and show that charter schools have higher average scores and passing rates than district schools.

In [ ]:
import pandas as pd
from pathlib import Path

In [96]:
#Files to load
schools_file = Path('../Resources/schools_complete.csv')
students_file = Path('../Resources/students_complete.csv')

#Read files and store into DataFrames
schools_df = pd.read_csv(schools_file)
students_df = pd.read_csv(students_file)

#Merge both dataframes into a single dataframe
complete_df = pd.merge(students_df, schools_df, left_on = 'school_name', right_on = 'school_name')

#Rename columns for readability
complete_df = complete_df.rename(columns = {'student_name': 'Student Name', 'gender': 'Gender', 'grade': 'Grade', 'school_name': 'School Name',
                                 'reading_score': 'Reading Score', 'math_score': 'Math Score', 'type': 'School Type', 'size': 'School Size', 'budget': 'School Budget'})
complete_df.head()

,Student ID,Student Name,Gender,Grade,School Name,Reading Score,Math Score,School ID,School Type,School Size,School Budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

In [97]:
#Get total number of unique schools
total_schools = complete_df['School Name'].nunique()
total_schools

15

In [99]:
#Get total number of students
total_students = complete_df['Student Name'].count()
total_students

39170

In [105]:
#Get the total budget for all schools
total_budget = schools_df['budget'].sum()
total_budget

24649428

In [107]:
#Get the average math score
avg_math_score = complete_df['Math Score'].mean()
avg_math_score

78.98537145774827

In [108]:
#Get the average reading score
avg_reading_score = complete_df['Reading Score'].mean()
avg_reading_score

81.87784018381414

In [109]:
#Code used from PyCitySchools_starter.ipynb to get percentage of students who passed math (score greater than or equal to 70)
passing_math_count = complete_df[(complete_df["Math Score"] >= 70)].count()["Student Name"]
passing_math_percentage = passing_math_count / float(total_students) * 100
passing_math_percentage

74.9808526933878

In [111]:
#Get percentage of students who passed reading (score greater than or equal to 70)
passing_reading_count = complete_df[(complete_df['Reading Score'] >= 70)].count()['Student Name']
passing_reading_percentage = passing_reading_count / float(total_students) * 100
passing_reading_percentage

85.80546336482001

In [112]:
#Code used from PyCitySchools_starter.ipynb to get percentage of students who passed math and reading
passing_math_reading_count = complete_df[
    (complete_df["Math Score"] >= 70) & (complete_df["Reading Score"] >= 70)
].count()["Student Name"]
overall_passing_rate = passing_math_reading_count /  float(total_students) * 100
overall_passing_rate

65.17232575950983

In [114]:
#Create high-level snapshot of district's key metrics in a DataFrame
district_summary = pd.DataFrame([{'Unique Schools': total_schools, 'Total Students': total_students, 'Total Budget': total_budget,
                                 'Average Math Score': avg_math_score, 'Average Reading Score': avg_reading_score, '% Passing Math': passing_math_percentage, 
                                 '% Passing Reading': passing_reading_percentage, '% Overall Passing': overall_passing_rate}])

#Formatting - code used from PyCitySchools_starter.ipynb
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

,Unique Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


# School Summary

In [239]:
#Get list of school names in alphabetical order
school_names = complete_df['School Name'].sort_values().unique()

In [240]:
#Get school types per school
school_types = complete_df.groupby('School Name')['School Type'].unique()
#Change data type to string and remove '[' and ']'
school_types = school_types.astype(str)
school_types = school_types.str.replace('[', '').str.replace(']', '')

/var/folders/m2/ysnn55010957_py0m5n5dds40000gn/T/ipykernel_92752/2233413270.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  school_types = school_types.str.replace('[', '').str.replace(']', '')


In [241]:
#Get number of students per school and set data type to integer
students_per_school = complete_df.groupby('School Name')['School Size'].unique()
students_per_school = students_per_school.astype(int)

In [242]:
#Get total school budget and set data type to integer
total_school_budget = complete_df.groupby('School Name')['School Budget'].unique()
total_school_budget = total_school_budget.astype(int)

In [243]:
#Get per student budget and set data type to integer
school_budget_per_student = total_school_budget / students_per_school
school_budget_per_student = school_budget_per_student.astype(int)

In [244]:
#Get average math score per school
school_avg_math_score = complete_df.groupby('School Name')['Math Score'].mean()

In [245]:
#Get average reading score per school
school_avg_reading_score = complete_df.groupby('School Name')['Reading Score'].mean()

In [246]:
#Get percentage of students per school who passed math (score equal to or greater than 70) and convert data type to float
students_passing_math = complete_df[complete_df['Math Score'] >= 70].groupby('School Name').count()['Student Name']
per_school_passing_math = (students_passing_math / students_per_school) * 100
per_school_passing_math = per_school_passing_math.astype('float')

In [247]:
#Get percentage of students per school who passed reading (score equal to or greater than 70) and convert data type to float
students_passing_reading = complete_df[complete_df['Reading Score'] >= 70].groupby('School Name').count()['Student Name']
per_school_passing_reading = (students_passing_reading / students_per_school) * 100
per_school_passing_reading = per_school_passing_reading.astype('float')

In [248]:
#Code used from PyCitySchools_starter.ipynb to get percentage of students who passed math and reading per school
students_passing_math_and_reading = complete_df[(complete_df["Reading Score"] >= 70) & (complete_df["Math Score"] >= 70)]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby("School Name").size()
overall_passing_rate = (school_students_passing_math_and_reading / students_per_school) * 100

In [186]:
#Create DataFrame to summarize key metrics about each school
schools_dict = {'School Type': school_types, 'Total Students': students_per_school, 
                                    'Total School Budget': total_school_budget, 'Per Student Budget': school_budget_per_student, 'Average Math Score': school_avg_math_score,
                                    'Average Reading Score': school_avg_reading_score, '% Passing Math': per_school_passing_math, '% Passing Reading': per_school_passing_reading,
                                    '% Overall Passing': overall_passing_rate}
per_school_summary = pd.DataFrame(schools_dict)

#Formatting as per code in PyCitySchools_starter.ipynb
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,'District',4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,'Charter',1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,'District',2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,'District',2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,'Charter',1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,'District',4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,'Charter',427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,'District',2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,'District',4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


# Highest-Performing Schools (by % Overall Passing)

In [187]:
#Sort schools by % Overall Passing in descending order and display the top 5 schools
top_schools = per_school_summary.sort_values('% Overall Passing', ascending = False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,'Charter',1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,'Charter',1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,'Charter',1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,'Charter',2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,'Charter',962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


# Lowest-Performing Schools (by % Overall Passing)

In [188]:
#Sort schools by % Overall Passing in ascending order and display the bottom 5 schools
bottom_schools = per_school_summary.sort_values('% Overall Passing')
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,'District',3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,'District',2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,'District',2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,'District',4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,'District',4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


# Math Scores by Grade
Perform the necessary calculations to create a DataFrame that lists the average math score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [192]:
#Code used from PyCitySchools_starter.ipynb to separate data by grade
ninth_graders = complete_df[(complete_df["Grade"] == "9th")]
tenth_graders = complete_df[(complete_df["Grade"] == "10th")]
eleventh_graders = complete_df[(complete_df["Grade"] == "11th")]
twelfth_graders = complete_df[(complete_df["Grade"] == "12th")]

#Get the average math score for each grade at each school
ninth_grade_math_scores = ninth_graders.groupby('School Name')['Math Score'].mean()
tenth_grade_math_scores = tenth_graders.groupby('School Name')['Math Score'].mean()
eleventh_grade_math_scores = eleventh_graders.groupby('School Name')['Math Score'].mean()
twelfth_grade_math_scores = twelfth_graders.groupby('School Name')['Math Score'].mean()

#Combine into a single dataframe
math_scores_by_grade_dict = {'9th': ninth_grade_math_scores, '10th': tenth_grade_math_scores, 
                                      '11th': eleventh_grade_math_scores, '12th': twelfth_grade_math_scores}
math_scores_by_grade = pd.DataFrame(math_scores_by_grade_dict)

#Minor data wrangling - Code used from PyCitySchools_starter.ipynb 
math_scores_by_grade.index.name = None

math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Scores by Grade
Create a DataFrame that lists the average reading score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [193]:
#Code used from PyCitySchools_starter.ipynb to separate data by grade
ninth_graders = complete_df[(complete_df["Grade"] == "9th")]
tenth_graders = complete_df[(complete_df["Grade"] == "10th")]
eleventh_graders = complete_df[(complete_df["Grade"] == "11th")]
twelfth_graders = complete_df[(complete_df["Grade"] == "12th")]

#Get the average reading score for each grade at each school
ninth_grade_reading_scores = ninth_graders.groupby('School Name')['Reading Score'].mean()
tenth_grade_reading_scores = tenth_graders.groupby('School Name')['Reading Score'].mean()
eleventh_grade_reading_scores = eleventh_graders.groupby('School Name')['Reading Score'].mean()
twelfth_grade_reading_scores = twelfth_graders.groupby('School Name')['Reading Score'].mean()

#Combine into a single dataframe
reading_scores_by_grade_dict = {'9th': ninth_grade_reading_scores, '10th': tenth_grade_reading_scores, 
                                      '11th': eleventh_grade_reading_scores, '12th': twelfth_grade_reading_scores}
reading_scores_by_grade = pd.DataFrame(reading_scores_by_grade_dict)

#Minor data wrangling - Code used from PyCitySchools_starter.ipynb 
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Scores by School Spending
Create a table that breaks down school performance based on average spending ranges (per student).

In [215]:
#Establish the bins - Code used from PyCitySchools_starter.ipynb 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [223]:
#Create a copy of the school summary since it has the "Per Student Budget" - Code used from PyCitySchools_starter.ipynb 
school_spending_df = per_school_summary.copy()

In [230]:
#First, remove the dollar sign from the Per Student Budget column 
school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].str.replace('$', '')
#Convert it from a string to a float
school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].astype('float')
#Use pd.cut to categorize spending based on the bin 
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['Per Student Budget'], spending_bins, labels = labels)
school_spending_df

/var/folders/m2/ysnn55010957_py0m5n5dds40000gn/T/ipykernel_92752/27375725.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].str.replace('$', '')


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
School Name,,,,,,,,,,
Bailey High School,'District',4976,"$3,124,928.00",628.0,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
Cabrera High School,'Charter',1858,"$1,081,356.00",582.0,83.061895,83.975780,94.133477,97.039828,91.334769,<$585
Figueroa High School,'District',2949,"$1,884,411.00",639.0,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Ford High School,'District',2739,"$1,763,916.00",644.0,77.102592,80.746258,68.309602,79.299014,54.289887,$630-645
Griffin High School,'Charter',1468,"$917,500.00",625.0,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630
Hernandez High School,'District',4635,"$3,022,020.00",652.0,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Holden High School,'Charter',427,"$248,087.00",581.0,83.803279,83.814988,92.505855,96.252927,89.227166,<$585
Huang High School,'District',2917,"$1,910,635.00",655.0,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Johnson High School,'District',4761,"$3,094,650.00",650.0,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680


In [232]:
#Calculate averages for math score, reading score, and overall score - Code used from PyCitySchools_starter.ipynb 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [233]:
#Create DataFrame to summarize the results
#First, create dictionary to hold data for dataframe
spending_summary_dict = {'Average Math Score': spending_math_scores, 'Average Reading Score': spending_reading_scores, '% Passing Math': spending_passing_math, 
                         '% Passing Reading': spending_passing_reading, '% Overall Passing': overall_passing_spending}
#Convert dictionary to dataframe
spending_summary = pd.DataFrame(spending_summary_dict)
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


# Scores by School Size
Use the per_school_summary DataFrame from the previous step to create a new DataFrame called type_summary. This new DataFrame should show school performance based on the "School Type".

In [234]:
#Establish the bins - Code used from PyCitySchools_starter.ipynb 
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Use pd.cut on Total Students column of per_school_summary DF
per_school_summary['School Size'] = pd.cut(per_school_summary['Total Students'], size_bins, labels = labels)
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
School Name,,,,,,,,,,
Bailey High School,'District',4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,'Charter',1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,'District',2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,'District',2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,'Charter',1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,'District',4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,'Charter',427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,'District',2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,'District',4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)


In [235]:
#Get average math score, reading score, and overall score - Code used from PyCitySchools_starter.ipynb 
#Group the per_school_summary DF by school size, then calculate the average scores for each category
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [236]:
#Create a DataFrame called size_summary that breaks down school performance based on school size (small, medium, or large)
#First, create a dictionary with the data to be contained in the dataframe
size_summary_dict = {'Average Math Score': size_math_scores, 'Average Reading Score': size_reading_scores, '% Passing Math': size_passing_math,
                    '% Passing Reading': size_passing_reading, '% Overall Passing': size_overall_passing}
#Convert dictionary to dataframe
size_summary = pd.DataFrame(size_summary_dict)
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


# Scores by School Type
Use the per_school_summary DataFrame from the previous step to create a new DataFrame called type_summary.
This new DataFrame should show school performance based on the "School Type".

In [237]:
#Get average math score, reading score, and overall score - Code used from PyCitySchools_starter.ipynb 
#Group the per_school_summary DF by school type, then calculate the average scores for each category
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [238]:
#Create a DataFrame called type_summary that breaks down school performance based on school type (Charter or District)
#First, create a dictionary with the data to be contained in the dataframe
type_summary_dict = {'Average Math Score': average_math_score_by_type, 'Average Reading Score': average_reading_score_by_type, 
                     '% Passing Math': average_percent_passing_math_by_type, '% Passing Reading': average_percent_passing_reading_by_type, 
                    '% Overall Passing': average_percent_overall_passing_by_type}
#Convert dictionary into dataframe
type_summary = pd.DataFrame(type_summary_dict)
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
'Charter',83.473852,83.896421,93.620830,96.586489,90.432244
'District',76.956733,80.966636,66.548453,80.799062,53.672208
